In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import psutil
import cv2

from model import CircleNet as CircleNet, CircleDataset
from pruning import CircleNet_P as CircleNet_P
from quantization import CircleNet_Q as CircleNet_Q
import pandas as pd
from torch.utils.data import DataLoader
# import jtop  

In [ ]:
def load_models():
    # 원본 모델 로드
    original_model = CircleNet()
    original_model.load_state_dict(torch.load('model/original_model.pth'))
    
    # Pruned 모델들 로드 (여러 ratio)
    pruned_models = {}
    for ratio in [0.3, 0.5, 0.7]:
        model = CircleNet_P(original_model, pruning_ratio=ratio)
        model.load_state_dict(torch.load(f'models/pruned_model_{int(ratio*100)}.pth'))
        pruned_models[f'pruned_{int(ratio*100)}'] = model
    
    # Quantized 모델들 로드 (8-bit, 4-bit)
    quantized_models = {}
    for bits in [8, 4]:
        model = CircleNet_Q()
        model.load_state_dict(torch.load(f'models/quantized_model_{bits}bit.pth'))
        quantized_models[f'quantized_{bits}bit'] = model
    
    return original_model, pruned_models, quantized_models

In [3]:
def measure_model_size(model):
    """모델 크기 측정 (파라미터 수, 메모리 사용량)"""
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    
    size_mb = (param_size + buffer_size) / 1024**2
    return {
        'parameters': sum(p.numel() for p in model.parameters()),
        'size_mb': size_mb
    }

In [4]:
def measure_inference_performance(model, test_loader, num_runs=100):
    """추론 성능 측정 (속도, 정확도, 메모리)"""
    model.eval()
    inference_times = []
    memory_usage = []
    accuracy = []
    
    with torch.no_grad():
        for _ in range(num_runs):
            for images, targets in test_loader:
                # 메모리 사용량 측정
                memory_usage.append(psutil.Process().memory_info().rss / 1024**2)
                
                # 추론 시간 측정
                start_time = time.time()
                outputs = model(images)
                inference_times.append((time.time() - start_time) * 1000)  # ms 단위
                
                # 정확도 계산 (예측값과 실제값의 차이가 1 이하인 경우 정확)
                pred = outputs.round()
                acc = (torch.abs(pred - targets) <= 1).float().mean().item()
                accuracy.append(acc)
    
    return {
        'avg_inference_time': np.mean(inference_times),
        'std_inference_time': np.std(inference_times),
        'avg_memory_usage': np.mean(memory_usage),
        'accuracy': np.mean(accuracy) * 100
    }

In [5]:
def measure_hardware_usage(model, test_loader):
    """Jetson Nano 하드웨어 사용량 측정"""
    with jtop() as jetson:
        stats = {
            'cpu_usage': [],
            'gpu_usage': [],
            'memory_usage': [],
            'power_usage': []
        }
        
        model.eval()
        with torch.no_grad():
            for images, _ in test_loader:
                _ = model(images)
                
                stats['cpu_usage'].append(jetson.cpu['cpu']['usage'])
                stats['gpu_usage'].append(jetson.gpu['gpu']['usage'])
                stats['memory_usage'].append(jetson.memory['used'] / jetson.memory['total'] * 100)
                stats['power_usage'].append(jetson.power['power'])
    
    return {k: np.mean(v) for k, v in stats.items()}

In [6]:
def plot_performance_comparison(results):
    """성능 비교 시각화"""
    # 결과를 DataFrame으로 변환
    df = pd.DataFrame(results)
    
    # 1. 메모리 vs 정확도
    plt.figure(figsize=(10, 6))
    plt.scatter(df['size_mb'], df['accuracy'])
    for i, model in enumerate(df.index):
        plt.annotate(model, (df['size_mb'][i], df['accuracy'][i]))
    plt.xlabel('Model Size (MB)')
    plt.ylabel('Accuracy (%)')
    plt.title('Model Size vs Accuracy Trade-off')
    plt.show()
    
    # 2. 추론 시간 비교
    plt.figure(figsize=(10, 6))
    plt.bar(df.index, df['avg_inference_time'])
    plt.xticks(rotation=45)
    plt.ylabel('Inference Time (ms)')
    plt.title('Average Inference Time Comparison')
    plt.show()
    
    # 3. 하드웨어 사용량 히트맵
    hardware_metrics = ['cpu_usage', 'gpu_usage', 'memory_usage', 'power_usage']
    hw_data = df[hardware_metrics]
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(hw_data.T, annot=True, fmt='.1f', cmap='YlOrRd')
    plt.title('Hardware Resource Usage Comparison')
    plt.show()

In [7]:
def main():
    # 모델 로드
    original_model, pruned_models, quantized_models = load_models()
    
    # 테스트 데이터 로드
    test_dataset = CircleDataset('test/img', 'test/target')
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    # 결과 저장할 딕셔너리
    results = {}
    
    # 1. 원본 모델 평가
    print("Evaluating original model...")
    results['original'] = {
        **measure_model_size(original_model),
        **measure_inference_performance(original_model, test_loader),
        **measure_hardware_usage(original_model, test_loader)
    }
    
    # 2. Pruned 모델들 평가
    for name, model in pruned_models.items():
        print(f"Evaluating {name}...")
        results[name] = {
            **measure_model_size(model),
            **measure_inference_performance(model, test_loader),
            **measure_hardware_usage(model, test_loader)
        }
    
    # 3. Quantized 모델들 평가
    for name, model in quantized_models.items():
        print(f"Evaluating {name}...")
        results[name] = {
            **measure_model_size(model),
            **measure_inference_performance(model, test_loader),
            **measure_hardware_usage(model, test_loader)
        }
    
    # 결과 시각화
    plot_performance_comparison(results)
    
    # 결과를 CSV로 저장
    pd.DataFrame(results).T.to_csv('model_comparison_results.csv')
    
    return results

In [8]:
if __name__ == "__main__":
    results = main()
    print("\nSummary of results:")
    print(pd.DataFrame(results).T)

C:\Users\ghddp\AppData\Local\Temp\ipykernel_28700\2566148516.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  original_model.load_state_dict(torch.load('CircleDetection\m

FileNotFoundError: [Errno 2] No such file or directory: 'CircleDetection\\model\\CircleNet.pth'